In [ ]:
# "Relationship between maternal serotonin levels and autism-associated genetic variants"
# 20240725

# Initialize environment

## Libraries

In [ ]:
PACKAGES <- rlang::quos(
    arm,
    here,
    boot,
    broom,
    e1071,
    EnvStats,
    fs,
    ggplot2,
    gt,
    gtsummary,
    janitor,
    patchwork,
    quantreg,
    readxl,
    skimr,
    tidyverse,
    WRS2)

lapply(PACKAGES, rlang::quo_name) |> lapply(library, character.only = TRUE) |>
    invisible()

message("Loaded libraries.")

## Paths

In [ ]:
ROOT <- path(
    'data', 
    'raw')

MASTER <- path(
    '0_MASTER',
    '_20231212_UIC_Vanderbilt_de_novo_SNV-InDel_CNV_MASTER_v4_SERT_excluded.xlsx')

MASTER_PRIOR <- path(
    '0_MASTER',
    '_2022-11-29_UIC_Vanderbilt_de_novo_SNV-InDel_CNV_MASTER_v4_pared_downloaded_20221201.xlsx')

SSC_DESCRIPTIVES <- path(
    '6_SSC_Version_15_Phenotype_Data_Set',
    'Proband_Data',
    'ssc_core_descriptive.csv')

ACE_DEMOGRAPHICS <- path(
    'UIC_ACE_Demo_2023-09-22.xlsx')

PGS <- path(
    'ssc_asd_pgs_euro_2022-04-18.csv')

VARIANT_DATA <- path(
    '2024-03-22_Supplemental_Table_v1_no_headings.xlsx')

## Constants

In [ ]:
QUANTILES <- c(0.10, 0.25, 0.50, 0.75, 0.90)

# p value threshold
THRESHOLD <- 2

X_LABELS <- c(
        "0.1%",
        "0.25%",
        "0.50%",
        "0.75%",
        "0.9%")

Y_LABELS <- c(
        "Latino/Latina ethnicity",
        "Asian race",
        "Black race",
        "White race",
        "Male sex",
        "Vanderbilt study site",
        "Maternal whole blood serotonin")

P_ROUND_FACTOR <- 3

# Import data

In [ ]:
df_master <- 
    read_excel(here(ROOT, MASTER), sheet = "Single row per Subject") |>

    rename(
        site = UIC_VAN,
        id = ID_Consolidated,
        relationship = Relationship,
        wb5ht = WB5HT,
        wb5ht_maternal = m_WB5HT,
        wb5ht_paternal = p_WB5HT,
        variant = Include_Variant) |>

    mutate(
        wb5ht_transformed = log10(wb5ht),
        wb5ht_maternal_transformed = log10(wb5ht_maternal))

df_master$site <- as_factor(df_master$site)

df_master$site <- fct_recode(df_master$site,
    "UIC" = "1",
    "Vanderbilt" = "2",
    "Vanderbilt" = "3",
    "UIC" = "4",
    "UIC" = "5")

df_ssc_descriptives <- 
    read_csv(here(ROOT, SSC_DESCRIPTIVES)) |>
    rename(id = individual) |>
    rename(age = age_at_ados)

df <- df_master |> 
    left_join(df_ssc_descriptives, by = "id") |>

    distinct(id, .keep_all = TRUE) |>

    mutate(
        race_white = if_else(race=="white", 1, 0),
        race_black = if_else(race=="african-amer", 1, 0),
        race_asian = if_else(race=="asian", 1, 0),
        race_other = if_else(race=="other", 1, 0),
        race_more_than_one = if_else(race=="more-than-one-race", 1, 0),
        race_not_specified = if_else(race=="not_specified", 1, 0),
    
        ethnicity_latin = if_else(ethnicity=="hispanic", 1, 0))

df_ace_demographics <- 
    read_excel(here(ROOT, ACE_DEMOGRAPHICS), sheet = "UIC_ACE_Demo_2023-09-22") |>

    rename(
        id = dbGaP_SAMPLE_ID,
        relationship_ace = Rel_to_Proband,
        age = Draw_Age_Months,
        sex_ace = Sex_numerical,
        ancestry_ace = Ancestry,
        ethnicity_ace = Ethnicity,
        race_ace = Race) |>

    select(-relationship_ace)

df_ace_demographics <- df_ace_demographics |>
    mutate(
        sex = if_else(sex_ace==1, "male", "female"),

        race_white = if_else(race_ace==1, 1, 0),
        race_black = if_else(race_ace==2, 1, 0),
        race_asian = if_else(race_ace==4, 1, 0),
        race_other = if_else(race_ace==5 | race_ace==6, 1, 0),
        race_more_than_one = if_else(race_ace==10, 1, 0),
        race_not_specified = if_else(race_ace==9, 1, 0),

        ethnicity_latin = if_else(ethnicity_ace==1, 1, 0)) |>

    select(-c(NDAR_GUID, Family_ID, sex_ace, ancestry_ace, ethnicity_ace, race_ace))

df <- df |> rows_update(df_ace_demographics, by = "id", unmatched = "ignore")

df_pgs <- 
    read_csv(here(ROOT, PGS)) |>
    
    rename(
        id = individual,
        relationship = fam)

# Set up dataframes

## Paternal

In [ ]:
df_genetic_paternal <- df |> filter(relationship=="Proband/ASD" & !is.na(wb5ht_paternal))

df_genetic_variant_paternal <- df_genetic_paternal |> filter(variant==1)

df_genetic_no_variant_paternal <- df_genetic_paternal |> filter(variant==0)

In [ ]:
rosnerTest(df_genetic_variant_paternal$wb5ht_paternal, k = 10)$all.stats

df_genetic_variant_paternal <- df_genetic_variant_paternal |> filter(wb5ht_paternal < 406.53691)

In [ ]:
rosnerTest(df_genetic_no_variant_paternal$wb5ht_paternal, k = 10)$all.stats

df_genetic_paternal <- bind_rows(df_genetic_variant_paternal, df_genetic_no_variant_paternal)

In [ ]:
df_genetic_paternal$variant <- as.factor(df_genetic_paternal$variant)

## Proband

In [ ]:
df_genetic_proband <- df |> filter(relationship=="Proband/ASD" & !is.na(wb5ht))

df_genetic_variant_proband <- df_genetic_proband |> filter(variant==1)

df_genetic_no_variant_proband <- df_genetic_proband |> filter(variant==0)

In [ ]:
rosnerTest(df_genetic_variant_proband$wb5ht, k = 10)$all.stats

df_genetic_variant_proband <- df_genetic_variant_proband |> filter(wb5ht < 665)

In [ ]:
rosnerTest(df_genetic_no_variant_proband$wb5ht, k = 10)$all.stats

df_genetic_no_variant_proband <- df_genetic_no_variant_proband |> filter(wb5ht < 609)

In [ ]:
df_genetic_proband_no_outliers <- bind_rows(df_genetic_variant_proband, df_genetic_no_variant_proband)

In [ ]:
df_genetic_proband_no_outliers$variant <- as.factor(df_genetic_proband_no_outliers$variant)

## Maternal

In [ ]:
df_genetic <- df |> filter(relationship=="Proband/ASD" & !is.na(wb5ht_maternal))

df_genetic_variant <- df_genetic |> filter(variant==1)

df_genetic_no_variant <- df_genetic |> filter(variant==0)

In [ ]:
rosnerTest(df_genetic_variant$wb5ht_maternal, k = 10)$all.stats

df_genetic_variant <- df_genetic_variant |> filter(wb5ht_maternal < 290.74000)

In [ ]:
rosnerTest(df_genetic_no_variant$wb5ht_maternal, k = 10)$all.stats

df_genetic_no_outliers <- bind_rows(df_genetic_variant, df_genetic_no_variant)

In [ ]:
df_genetic_no_outliers$variant <- as_factor(df_genetic_no_outliers$variant)

# Get variant data

In [ ]:
df_variant_data <- 
    read_excel(here(ROOT, VARIANT_DATA), sheet = "Supplemental Table 1") |>
    rename(
        id = ID_Consolidated)

In [ ]:
df_variant_data <- df_variant_data %>%
  mutate(id = str_replace_all(id, "_", "@"))

In [ ]:
df_variant_data_variants <- df_variant_data |> filter(Include_Variant1==1)

In [ ]:
df_variant_data_variants |> select('id', 'SNV gene', 'CNV', 'DelDup', 'Inheritance', 'Maternal WB5-HT (ng/mL)')

In [ ]:
df_genetic_with_variant_data <- df_genetic_no_outliers |> left_join(df_variant_data, by = "id")

In [ ]:
df_variants <- df_genetic_with_variant_data |> filter(variant==1)

In [ ]:
df_variants_index <- df_variants |> select('SNV gene', 'CNV', 'DelDup', 'Inheritance')

In [ ]:
cnvs <- df_variants_index |> select('CNV', 'DelDup', 'Inheritance')

In [ ]:
cnvs |> write_csv('cnvs.csv')

In [ ]:
snvs <- df_variants_index |> select('SNV gene', 'Inheritance')

In [ ]:
snvs |> write_csv('snvs.csv')

# Figure

## Setup

In [ ]:
df_genetic_proband_no_outliers$wb5ht |> range()

df_genetic_paternal$wb5ht_paternal |> range()

df_genetic_no_outliers$wb5ht_maternal |> range()

In [ ]:
YMIN <- 51.54

YMAX <- 545

In [ ]:
levels(df_genetic_no_outliers$variant) <- c("Non-carrier", "Rare variant carrier")

levels(df_genetic_proband_no_outliers$variant) <- c("Non-carrier", "Rare variant carrier")

levels(df_genetic_paternal$variant) <- c("Non-carrier", "Rare variant carrier")

In [ ]:
df_non_variants <- df_genetic_no_outliers |> filter(variant=="Non-carrier")

In [ ]:
df_variants <- df_genetic_no_outliers |> filter(variant=="Rare variant carrier")

In [ ]:
cor.test(df_variants$wb5ht_maternal_transformed, df_variants$wb5ht_transformed, method = "pearson") |> tidy()

In [ ]:
cor.test(df_non_variants$wb5ht_maternal_transformed, df_non_variants$wb5ht_transformed, method = "pearson") |> tidy()

## Build panels

### A (blank)

In [ ]:
df_conceptual <- df_genetic_no_outliers

df_conceptual <- df_conceptual |> mutate(non_control = 1)

df_conceptual$non_control <- as.factor(df_conceptual$non_control)

levels(df_conceptual$non_control) <- c("Autistic offspring", "Non-autistic offspring (hypothetical)")

a_plot <- ggplot() + 
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    labs(
        y = "",
        x = "") +
    ylim(YMIN, YMAX)

### B (maternal)

In [ ]:
b_plot <- ggplot(df_genetic_no_outliers, aes(x=variant, y=wb5ht_maternal)) + 
    geom_violin(
        alpha = 0.5, 
        aes(fill = variant), 
        draw_quantiles = c(0.10, 0.25, 0.50, 0.75, 0.90),
        adjust=1,
        scale="width",
        trim=TRUE) +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5, size = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          plot.margin = unit(c(0,0,0,0), "cm")) +
    labs(
        y = "Maternal WB5-HT (ng/ml)",
        x = "Proband genetic status") +
    scale_fill_brewer(palette="Set1") +
    ylim(YMIN, YMAX)


In [ ]:
datapoints <- df_genetic_no_outliers |> select(id, variant, wb5ht_maternal)

In [ ]:
datapoints <- datapoints |> mutate(id = as.character(row_number()))

In [ ]:
datapoints_b <- datapoints |> mutate(panel = "B")

### C (proband)

In [ ]:
c_plot <- ggplot(df_genetic_proband_no_outliers, aes(x=variant, y=wb5ht)) + 
    geom_violin(
        alpha = 0.5, 
        aes(fill = variant), 
        draw_quantiles = c(0.10, 0.25, 0.50, 0.75, 0.90),
        scale="width",
        adjust=1,
        trim=TRUE) +    
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5, size = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        y = "Proband WB5-HT (ng/ml)",
        x = "Proband genetic status") +
    scale_fill_brewer(palette="Set1") +
    ylim(YMIN, YMAX)

In [ ]:
datapoints <- df_genetic_proband_no_outliers |> select(id, variant, wb5ht)

In [ ]:
datapoints <- datapoints |> mutate(id = as.character(row_number()))

In [ ]:
datapoints_c <- datapoints |> mutate(panel = "C")

### D (paternal)

In [ ]:
d_plot <- ggplot(df_genetic_paternal, aes(x=variant, y=wb5ht_paternal)) + 
 geom_violin(
        alpha = 0.5, 
        aes(fill = variant), 
        draw_quantiles = c(0.10, 0.25, 0.50, 0.75, 0.90),
        scale="width",
        adjust=1,
        trim=TRUE) +    
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5, size = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        y = "Paternal WB5-HT (ng/ml)",
        x = "Proband genetic status") +
    scale_fill_brewer(palette="Set1") +
    ylim(YMIN, YMAX)

In [ ]:
datapoints <- df_genetic_paternal |> select(id, variant, wb5ht_paternal)

In [ ]:
datapoints <- datapoints |> mutate(id = as.character(row_number()))

In [ ]:
datapoints_d <- datapoints |> mutate(panel = "D")

In [ ]:
datapoints <- bind_rows(datapoints_b, datapoints_c, datapoints_d)

In [ ]:
datapoints <- datapoints |> select(id, variant, wb5ht_maternal, wb5ht, wb5ht_paternal, panel)

In [ ]:
datapoints <- datapoints |> rename(
    variant_status = variant,
    wb5ht_proband = wb5ht)

In [ ]:
write_csv(datapoints, "datapoints.csv")


## Assemble panels

In [ ]:
panels_plot <- a_plot + b_plot + c_plot + d_plot

panels_plot <- panels_plot + plot_layout(ncol = 2, nrow = 2)

panels_plot <- panels_plot + plot_annotation(tag_levels = 'A')

panels_plot

ggsave("panels.png", width = 10, height = 10, dpi = 300)

# Supplement 1

In [ ]:
df_genetic_no_outliers_pristine |> nrow()

In [ ]:
df_genetic_no_outliers_pristine |> group_by(variant) |> count()

In [ ]:
df_genetic_no_outliers_pristine |> colnames()

In [ ]:
df_genetic_no_outliers_pristine |> 
    select("id", "variant") |>
    write_csv("20231220_participants.csv")

In [ ]:
df_genetic_no_outliers <- df_genetic_no_outliers_pristine

In [ ]:
df_1 <- df_genetic_no_outliers |> filter(variant==1)

In [ ]:
df_2 <- df_genetic_no_outliers |> filter(variant==0)

In [ ]:
densities <- density_1 + density_2

densities

ggsave("densities.png", width = 10, height = 10, dpi = 300)

In [ ]:
shapiro.test(df_1$wb5ht_maternal)

In [ ]:
shapiro.test(df_2$wb5ht_maternal)

In [ ]:
skewness(df_1$wb5ht_maternal)

In [ ]:
skewness(df_2$wb5ht_maternal)

In [ ]:
range(df_1$wb5ht_maternal)

In [ ]:
range(df_2$wb5ht_maternal)

In [ ]:
datapoints_3 <- df_1 |> select(id, wb5ht_maternal) |>
    mutate(variant_status="Rare variant carrier") |>
    mutate(id = as.character(row_number())) |>
    select(id, variant_status, wb5ht_maternal)

In [ ]:
datapoints_4 <- df_2 |> select(id, wb5ht_maternal) |>
    mutate(variant_status="Non-carrier") |>
    mutate(id = as.character(row_number())) |>
    select(id, variant_status, wb5ht_maternal)

In [ ]:
datapoints_3 <- bind_rows(datapoints_3, datapoints_4)

In [ ]:
datapoints_3 |> mutate(id = as.character(row_number()))

In [ ]:
write_csv(datapoints_3, "datapoints_3.csv")

In [ ]:
# Create density plot
ggplot(df_1, aes(x = wb5ht_maternal)) + 
  geom_density(fill="blue", alpha=0.5) + 
  labs(title="Maternal WB5-HT distribution among rare variant carriers", x="Maternal WB5-HT", y="Density") +
  theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    xlim(58, 369)

In [ ]:
ggsave("density_1.png")

In [ ]:
# Create density plot
ggplot(df_2, aes(x = wb5ht_maternal)) + 
  geom_density(fill="red", alpha=0.5) + 
  labs(title="Maternal WB5-HT distribution among non-carriers", x="Maternal WB5-HT", y="Density") +
  theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    xlim(58, 369)

In [ ]:
ggsave("density_2.png")

In [ ]:
# Create density plot
ggplot(df_1, aes(x = wb5ht_maternal)) + 
  geom_density(fill="blue", alpha=0.5) + 
  labs(title="Maternal WB5-HT distribution among rare variant carriers", x="Maternal WB5-HT", y="Density")


In [ ]:
data1 <- df_1$wb5ht_maternal
data2 <- df_2$wb5ht_maternal

set.seed(1)  
bootstrap1 <- boot(data1, skewness_function_e1071, R=10000)
bootstrap2 <- boot(data2, skewness_function_e1071, R=10000)

ci1 <- boot.ci(bootstrap1, type="perc")
ci2 <- boot.ci(bootstrap2, type="perc")

ci1_lower_bound <- ci1$percent[4]

ci1_upper_bound <- ci1$percent[5]
ci2_lower_bound <- ci2$percent[4]

ci2_upper_bound <- ci2$percent[5]

if (ci1_lower_bound < ci2_upper_bound) {
  print("skewness of data2 > data1.")
} else {
  print("skewness of data2 not > data1.")
}

ci1
ci2 

In [ ]:
ci_upper_data2

In [ ]:
ci_lower_data1

## Compare medians

In [ ]:
wilcox.exact(wb5ht_paternal ~ variant, df_genetic_paternal)

In [ ]:
wilcox.exact(wb5ht ~ variant, df_genetic_proband_no_outliers)

In [ ]:
wilcox.exact(wb5ht_maternal ~ variant, df_genetic_no_outliers)

In [ ]:
rare_vs_non_paternal <- qcomhd(wb5ht_paternal ~ variant, df_genetic_paternal)

## Compare across quantiles

In [ ]:
set.seed(1)

In [ ]:
rare_vs_non_proband <- qcomhd(wb5ht ~ variant, df_genetic_proband_no_outliers)

In [ ]:
rare_vs_non_proband

In [ ]:
rare_vs_non_paternal <- qcomhd(wb5ht_paternal ~ variant, df_genetic_paternal)

In [ ]:
rare_vs_non_paternal

In [ ]:
rare_vs_non <- qcomhd(wb5ht_maternal ~ variant, df_genetic_no_outliers)

In [ ]:
rare_vs_non

In [ ]:
df_genetic_no_outliers <- df_genetic_no_outliers |> mutate(
    condition = "Mother")

df_genetic_no_outliers$condition <- as_factor(df_genetic_no_outliers$condition)

In [ ]:
df_genetic_proband_no_outliers <- df_genetic_proband_no_outliers |> mutate(
    condition = "Proband")

df_genetic_proband_no_outliers$condition <- as_factor(df_genetic_proband_no_outliers$condition)

In [ ]:
df_genetic_no_outliers <- df_genetic_no_outliers |> mutate(
    sert = wb5ht_maternal)

In [ ]:
df_genetic_proband_no_outliers <- df_genetic_proband_no_outliers |> mutate(
    sert = wb5ht)

In [ ]:
combined <- df_genetic_proband_no_outliers |> bind_rows(df_genetic_no_outliers)

In [ ]:
ggplot(combined, aes(x=condition, y=sert))+ 
    geom_boxplot(
        alpha = 0.5, 
        aes(fill = condition)) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Whole blood serotonin in probands vs. mothers",
        y = "Whole blood serotonin (ng/ml)",
        x = "Proband or mother") +
    scale_fill_brewer(palette="Set1")

## Generate tables

In [ ]:
df_genetic_no_outliers <- df_genetic_no_outliers |> mutate(
    age_years = age/12)

In [ ]:
df_genetic_no_outliers_age <- df_genetic_no_outliers |> filter(!is.na(age))

In [ ]:
df_table <- df_genetic_no_outliers |> 
    mutate(race = case_when(
        race_white == "1" ~ "white",
        race_black == "1" ~ "african-amer",
        race_asian == "1" ~ "asian",
        race_more_than_one == "1" ~ "more-than-one-race",
        race_not_specified == "1" ~ "not-specified",
    ))

df_table$race <- recode_factor(df_table$race, 
    `african-amer` = "Black", 
    `white` = "White",
    `asian` = "Asian",
    `more-than-one-race` = "More than one race",
    `not-specified` = "Not specified")

In [ ]:
df_table <- df_table |> 
    mutate(ethnicity_merged = case_when(
        ethnicity == "non-hispanic" ~ "non-hispanic",
        ethnicity == "hispanic" ~ "hispanic",
        ethnicity_latin == "1" ~ "hispanic",
        ethnicity_latin == "0" ~ "non-hispanic"))


In [ ]:
df_table$sex <- recode_factor(df_table$sex, 
    male = "Male", 
    female = "Female")

In [ ]:
df_table$ethnicity_merged <- recode_factor(df_table$ethnicity_merged, 
    hispanic = "Latine", 
    `non-hispanic` = "Non-Latine")

In [ ]:
df_table$variant <- recode_factor(df_table$variant, 
    `0` = "Non-carrier", 
    `1` = "Carrier")

In [ ]:
df_table_freeze <- df_table

In [ ]:
df_table <- df_table |> 
    select(
        age_years,
        site, 
        sex,
        race,
        variant,
        ethnicity_merged,
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
    ) 

table1 <- df_table |> tbl_summary(
    by = variant,
    type = list(
        age_years ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age_years,
        site,
        sex,
        race,
        ethnicity_merged),
    label = list(
        age_years ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity_merged ~ "Ethnicity"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants by rare variant carrier status**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1_gen.html"))

In [ ]:
table3 <- df_table |> tbl_summary(
    by = variant,
    type = list(
        ados_sa_css ~ "continuous",
        ados_rrb_css ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
        ados_sa_css,
        ados_rrb_css,
        adi_r_soc_a_total,
        adi_r_comm_b_non_verbal_total,
        adi_r_rrb_c_total,
        vineland_ii_composite_standard_score,
        ssc_diagnosis_nonverbal_iq),
    label = list(
        wb5ht ~ "Proband WB5-HT",
        wb5ht_maternal ~ "Maternal WB5-HT",
        wb5ht_paternal ~ "Paternal WB5-HT",
        ados_sa_css ~ "ADOS SA CSS",
        ados_rrb_css ~ "ADOS RRB CSS",
        adi_r_soc_a_total ~ "ADI-R domain A",
        adi_r_comm_b_non_verbal_total ~ "ADI-R domain B",
        adi_r_rrb_c_total ~ "ADI-R domain C",
        vineland_ii_composite_standard_score ~ "VABS-II composite",
        ssc_diagnosis_nonverbal_iq ~ "Nonverbal IQ"),
    missing = "no") |>
    add_overall(last = TRUE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_header(
        label = '**Measure**') |>
    modify_spanning_header(
        all_stat_cols() ~ "**WB5-HT levels and behavioral measure scores of study participants by rare variant carrier status**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table3.html"))

In [ ]:
table3 <- table_df |> tbl_summary(
    by = variant,
    type = list(
        ados_sa_css ~ "continuous",
        ados_rrb_css ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
        ados_sa_css,
        ados_rrb_css,
        adi_r_soc_a_total,
        adi_r_comm_b_non_verbal_total,
        adi_r_rrb_c_total,
        vineland_ii_composite_standard_score,
        ssc_diagnosis_nonverbal_iq),
    label = list(
        wb5ht ~ "Proband WB5-HT",
        wb5ht_maternal ~ "Maternal WB5-HT",
        wb5ht_paternal ~ "Paternal WB5-HT",
        ados_sa_css ~ "ADOS SA CSS",
        ados_rrb_css ~ "ADOS RRB CSS",
        adi_r_soc_a_total ~ "ADI-R domain A",
        adi_r_comm_b_non_verbal_total ~ "ADI-R domain B",
        adi_r_rrb_c_total ~ "ADI-R domain C",
        vineland_ii_composite_standard_score ~ "VABS-II composite",
        ssc_diagnosis_nonverbal_iq ~ "Nonverbal IQ"),
    missing = "no") |>
    add_overall(last = TRUE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_header(
        label = '**Measure**') |>
    modify_spanning_header(
        all_stat_cols() ~ "**WB5-HT levels and behavioral measure scores of study participants by variant carrier status**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table_gen.html"))

In [ ]:
ggplot(df_genetic_no_outliers_age, aes(x=variant, y=wb5ht_maternal))+ 
    geom_violin(
        alpha = 0.5, 
        aes(color = variant, fill = variant),
        scale = "area") +
    geom_violin(
        alpha = 0, 
        aes(),
        draw_quantiles = c(0.2, 0.4, 0.6, 0.8),
        scale = "area") +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Maternal whole blood serotonin by genetic status",
        y = "Whole blood serotonin (ng/ml)",
        x = "Offspring genetic status") +
    scale_fill_brewer(palette="Set1")

In [ ]:
ggplot(df_genetic_no_outliers_age, aes(x=variant, y=wb5ht_paternal)) + 
    geom_boxplot(
        alpha = 0.5, 
        aes(fill = variant)) +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Paternal whole blood serotonin by offspring genetic status",
        y = "Whole blood serotonin (ng/ml)",
        x = "Offspring genetic status") +
    scale_fill_brewer(palette="Set1")


In [ ]:
ggplot(df_genetic_no_outliers, aes(x=variant, y=wb5ht_maternal)) + 
    geom_boxplot(
        alpha = 0.5, 
        aes(fill = variant)) +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Maternal whole blood serotonin by offspring genetic status",
        y = "Whole blood serotonin (ng/ml)",
        x = "Offspring genetic status") +
    scale_fill_brewer(palette="Set1")


In [ ]:
df_genetic_no_outliers_age <- df_genetic_no_outliers_age |> mutate(wb5ht_paternal_transformed = log10(wb5ht_paternal))

In [ ]:
table_df <- df_genetic_no_outliers_age |> 
    select(
        age_years,
        variant,
        site, 
        sex,
        race,
        ethnicity,
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
        ados_sa_css,
        ados_rrb_css,
        adi_r_soc_a_total,
        adi_r_comm_b_non_verbal_total,
        adi_r_rrb_c_total,
        vineland_ii_composite_standard_score,
        ssc_diagnosis_nonverbal_iq
    ) 

table_df$race <- recode_factor(table_df$race, 
    `african-amer` = "Black", 
    white = "White",
    asian = "Asian",
    `more-than-one-race` = "More than one race",
    `not-specified` = "Not specified")

table_df$sex <- recode_factor(table_df$sex, 
    male = "Male", 
    female = "Female")

table_df$ethnicity <- recode_factor(table_df$ethnicity, 
    hispanic = "Latino/Latina", 
    `non-hispanic` = "Non-Latino/Latina")

table1 <- table_df |> tbl_summary(
    by = variant,
    type = list(
        age_years ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age_years,
        site,
        sex,
        race,
        ethnicity),
    label = list(
        age_years ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity ~ "Ethnicity"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants by rare variant carrier status**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1.html"))

In [ ]:
df_genetic$variant <- as_factor(df_genetic$variant)

In [ ]:
ggplot(df_genetic, aes(x=variant, y=wb5ht))+ 
    geom_violin(
        alpha = 0.5, 
        aes(color = variant, fill = variant),
        scale = "area") +
    geom_violin(
        alpha = 0, 
        aes(),
        draw_quantiles = c(0.2, 0.4, 0.6, 0.8),
        scale = "area") +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Maternal whole blood serotonin by genetic status",
        y = "Whole blood serotonin (ng/ml)",
        x = "Offspring genetic status") +
    scale_fill_brewer(palette="Set1")

In [ ]:
ggplot(df_genetic, aes(x=variant, y=wb5ht_maternal))+ 
    geom_violin(
        alpha = 0.5, 
        aes(color = variant, fill = variant),
        scale = "area") +
    geom_violin(
        alpha = 0, 
        aes(),
        draw_quantiles = c(0.2, 0.4, 0.6, 0.8),
        scale = "area") +
    geom_point(position = position_jitter(seed = 1, width = 0.2), alpha = 0.5) +
    theme(legend.position = "none",
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank()) +
    labs(
        title = "Maternal whole blood serotonin by genetic status",
        y = "Whole blood serotonin (ng/ml)",
        x = "Offspring genetic status") +
    scale_fill_brewer(palette="Set1")

In [ ]:
mat_variant_vs_non <- qcomhd(wb5ht_maternal ~ variant, df_genetic_no_outliers)

In [ ]:
pat_variant_vs_non <- qcomhd(wb5ht_paternal ~ variant, df_genetic_no_outliers)

In [ ]:
pro_variant_vs_non <- qcomhd(wb5ht ~ variant, df_genetic_no_outliers)

In [ ]:
mat_variant_vs_non

In [ ]:
mat_rare_vs_non_table <- mat_variant_vs_non$partable |>
    select(-c(p.crit)) |>
    gt() |>
    tab_header(
        title = "Maternal WB5-HT by quantile for those probands with a rare variant vs. those without") |>
    cols_label(
        q = "QU",
        n1 = md("*N* (no variant)"),
        n2 = md("*N* (variant)"),
        est1 = "Median (no variant)",
        est2 = "Median (variant)",
        "est1-est.2" = "Difference",
        ci.low = "CI Lower",
        ci.up = "CI Upper",
        p.value = md("*Adj. p*")) |>
    fmt_number(
        columns = 4:8,
        decimals = 2) |>
    fmt_number(
        columns = 9,
        decimals = 3) |>
    cols_align(
        align = "right",
        columns = everything()) |>
    tab_style(
        style = cell_text(weight = "bold"),
        locations = cells_body(
            rows = p.value < 0.05))

In [ ]:
rare_vs_non_proband <- qcomhd(wb5ht ~ variant, df_genetic_proband_no_outliers)

In [ ]:
pat_rare_vs_non_table <- pat_variant_vs_non$partable |>
    select(-c(p.crit)) |>
    gt() |>
    tab_header(
        title = "Paternal WB5-HT by quantile for those probands with a rare variant vs. those without") |>
    cols_label(
        q = "QU",
        n1 = md("*N* (no variant)"),
        n2 = md("*N* (variant)"),
        est1 = "Median (no variant)",
        est2 = "Median (variant)",
        "est1-est.2" = "Difference",
        ci.low = "CI Lower",
        ci.up = "CI Upper",
        p.value = md("*Adj. p*")) |>
    fmt_number(
        columns = 4:8,
        decimals = 2) |>
    fmt_number(
        columns = 9,
        decimals = 3) |>
    cols_align(
        align = "right",
        columns = everything()) |>
    tab_style(
        style = cell_text(weight = "bold"),
        locations = cells_body(
            rows = p.value < 0.05))

In [ ]:
pro_rare_vs_non_table <- rare_vs_non_proband$partable |>
    select(-c(p.crit)) |>
    gt() |>
    tab_header(
        title = "Proband WB5-HT by quantile for those probands with a rare variant vs. those without") |>
    cols_label(
        q = "QU",
        n1 = md("*N* (no variant)"),
        n2 = md("*N* (variant)"),
        est1 = "Median (no variant)",
        est2 = "Median (variant)",
        "est1-est.2" = "Difference",
        ci.low = "CI Lower",
        ci.up = "CI Upper",
        p.value = md("*Adj. p*")) |>
    fmt_number(
        columns = 4:8,
        decimals = 2) |>
    fmt_number(
        columns = 9,
        decimals = 3) |>
    cols_align(
        align = "right",
        columns = everything()) |>
    tab_style(
        style = cell_text(weight = "bold"),
        locations = cells_body(
            rows = p.value < 0.05))

In [ ]:
pro_rare_vs_non_table  |> 
    gtsave(here("output","pro_rare_vs_non_quantile_2.html"))

In [ ]:
pat_rare_vs_non_table  |> 
    gtsave(here("output","pat_rare_vs_non_quantile.html"))

In [ ]:
mat_rare_vs_non_table  |> 
    gtsave(here("output","mat_rare_vs_non_quantile.html"))

In [ ]:
wilcox.exact(wb5ht_maternal ~ variant, df_genetic_no_outliers_age)

In [ ]:
wilcox.exact(wb5ht_maternal ~ variant, df_genetic_no_outliers)

In [ ]:
wilcox.exact(wb5ht_maternal ~ variant, df)

In [ ]:
variant_vs_non <- qcomhd(wb5ht_maternal ~ variant, df_genetic_no_outliers_age)

In [ ]:
variant_vs_non

In [ ]:
variant_vs_non <- qcomhd(wb5ht_maternal ~ variant, df_genetic)

In [ ]:
variant_vs_non

In [ ]:
variant_vs_non <- qcomhd(wb5ht_maternal ~ variant, df_genetic_no_outliers)

In [ ]:
variant_vs_non

In [ ]:
model <- lm(
    ados_sa_css ~ 
        wb5ht_maternal + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- glm(wb5ht ~ variant, df_probands_5ht_no_outliers)

mylogit <- glm(admit ~ gre + gpa + rank, data = mydata, family = "binomial")


# Polygenic score analyses

In [ ]:
df_genetic_no_outliers |> filter(site=="UIC" & !is.na(sex)) |> nrow()

In [ ]:
df_pgs_score <- df_pgs |> select(id, SCORE, PC1, PC2, PC3, PC4, PC5, PC6, PC7, PC8, PC9, PC10)

In [ ]:
df_genetic_pgs <- df_genetic_no_outliers |> left_join(df_pgs_score, by = "id")

In [ ]:
df_genetic_pgs |> filter(!is.na(SCORE)) |> nrow()

In [ ]:
df_genetic_pgs <- df_genetic_pgs |> mutate(score_rescaled = rescale(SCORE))

In [ ]:
hist(df_genetic_pgs$SCORE, na.rm = TRUE)

In [ ]:
hist(df_core_pgs$score_rescaled, na.rm = TRUE)

In [ ]:
ggplot(df_core_pgs, aes(x=wb5ht_maternal_transformed, y=SCORE)) + geom_point()

In [ ]:
ggplot(df_core_pgs, aes(x=wb5ht_transformed, y=SCORE)) + geom_point()

In [ ]:
ggplot(data=df_genetic_pgs, aes(x=wb5ht_maternal_transformed, y=SCORE)) +
    geom_point() +
    labs(
        title = "Log-transformed maternal WB5-HT vs. autism polygenic score",
        y = "Autism polygenic score",
        x = "Log-transformed maternal WB5-HT")

In [ ]:
ggsave("wb5ht_vs_pgs.png")

In [ ]:
model <- lm(wb5ht_maternal_transformed ~
        SCORE,
        data = df_genetic_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(wb5ht_transformed ~
        score_rescaled,
        data = df_core_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(wb5ht_maternal_transformed ~
        score_rescaled,
        data = df_core_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(wb5ht_maternal_transformed ~
        score_rescaled + 
        site, 
        data = df_core_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(wb5ht_transformed ~
        score_rescaled + 
        site +
        sex, 
        data = df_core_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
model <- lm(wb5ht_maternal_transformed ~
        score_rescaled + 
        site +
        sex, 
        data = df_core_pgs)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
range(df_core_pgs$SCORE, na.rm = TRUE)

In [ ]:
rq_fit <- rq(
    wb5ht_paternal ~ score_rescaled, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht_maternal ~ SCORE, 
    data = df_genetic_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht ~ 
    score_rescaled, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht ~ 
    score_rescaled + site, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht ~ 
    score_rescaled + site + sex, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht_maternal ~ 
    score_rescaled + site, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht ~ score_rescaled + site + sex, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht_paternal ~ score_rescaled + site + sex, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
rq_fit <- rq(
    wb5ht_maternal ~ score_rescaled + site + sex, 
    data = df_core_pgs, 
    tau = QUANTILES)

fit_coefficients <- summary(rq_fit, se="ker")
fit_coefficients


In [ ]:
fit_coefficients <- summary(rq_fit, se="ker")

In [ ]:
model <- lm(
    wb5ht_maternal_transformed ~ SCORE,
    data = df_core_pgs)

summary(model)
tidy(model, conf.int = TRUE)

In [ ]:
lm(df_core_pgs ~
   
   
   model <- lm(
    ados_sa_css ~ 
        wb5ht_maternal + 
        site +
        sex +
        race_white +
        race_black +
        race_asian +
        ethnicity_latin,
    data = df_core)

summary(model)
tidy(model, conf.int=TRUE)

In [ ]:
df_pgs$fam |> unique()

# Table 1: Descriptive stats

In [ ]:
df_core <- df_genetic_no_outliers

table_df <- df_core |> 
    select(
        age,
        variant,
        site, 
        sex,
        race,
        ethnicity,
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
    ) 

table_df$race <- recode_factor(table_df$race, 
    `african-amer` = "Black", 
    white = "White",
    asian = "Asian",
    `more-than-one-race` = "More than one race",
    `not-specified` = "Not specified")

table_df$sex <- recode_factor(table_df$sex, 
    male = "Male", 
    female = "Female")

table_df$ethnicity <- recode_factor(table_df$ethnicity, 
    hispanic = "Latine", 
    `non-hispanic` = "Non-Latine")


In [ ]:
table_df <- table_df |> select(-wb5ht_maternal, -wb5ht_paternal)

In [ ]:
datapoints_2 <- table_df |> mutate(id = as.character(row_number())) |>
    select(id, age, variant, site, sex, race, ethnicity) |>
    rename(age_months = age)

In [ ]:
write_csv(datapoints_2, "datapoints_2.csv")

In [ ]:
df_core <- df_genetic_no_outliers

table_df <- df_core |> 
    select(
        age,
        variant,
        site, 
        sex,
        race,
        ethnicity,
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
    ) 

table_df$race <- recode_factor(table_df$race, 
    `african-amer` = "Black", 
    white = "White",
    asian = "Asian",
    `more-than-one-race` = "More than one race",
    `not-specified` = "Not specified")

table_df$sex <- recode_factor(table_df$sex, 
    male = "Male", 
    female = "Female")

table_df$ethnicity <- recode_factor(table_df$ethnicity, 
    hispanic = "Latine", 
    `non-hispanic` = "Non-Latine")

table1 <- table_df |> tbl_summary(
    by = variant,
    type = list(
        age ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age,
        site,
        sex,
        race,
        ethnicity),
    label = list(
        age ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity ~ "Ethnicity"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants by variant status**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1.html"))

In [ ]:
table3 <- table_df |> tbl_summary(
    by = class,
    type = list(
        ados_sa_css ~ "continuous",
        ados_rrb_css ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        wb5ht,
        wb5ht_maternal,
        wb5ht_paternal,
        ados_sa_css,
        ados_rrb_css,
        adi_r_soc_a_total,
        adi_r_comm_b_non_verbal_total,
        adi_r_rrb_c_total,
        vineland_ii_composite_standard_score,
        ssc_diagnosis_nonverbal_iq),
    label = list(
        wb5ht ~ "Proband WB5-HT",
        wb5ht_maternal ~ "Maternal WB5-HT",
        wb5ht_paternal ~ "Paternal WB5-HT",
        ados_sa_css ~ "ADOS SA CSS",
        ados_rrb_css ~ "ADOS RRB CSS",
        adi_r_soc_a_total ~ "ADI-R domain A",
        adi_r_comm_b_non_verbal_total ~ "ADI-R domain B",
        adi_r_rrb_c_total ~ "ADI-R domain C",
        vineland_ii_composite_standard_score ~ "VABS-II composite",
        ssc_diagnosis_nonverbal_iq ~ "Nonverbal IQ"),
    missing = "no") |>
    add_overall(last = TRUE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_header(
        label = '**Measure**') |>
    modify_spanning_header(
        all_stat_cols() ~ "**WB5-HT levels and behavioral measure scores of study participants by class**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table3.html"))

In [ ]:
table <- table_df |> tbl_summary(
    by = class,
    type = list(
        ados_sa_css ~ "continuous",
        ados_rrb_css ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    label = list(
        age ~ "Age in years",
        site ~ "Study site",
        sex ~ "Sex",
        race ~ "Race",
        ethnicity ~ "Ethnicity",
        wb5ht ~ "Proband WB5-HT",
        wb5ht_maternal ~ "Maternal WB5-HT",
        wb5ht_paternal ~ "Paternal WB5-HT",
        ados_sa_css ~ "ADOS SA CSS",
        ados_rrb_css ~ "ADOS RRB CSS",
        adi_r_soc_a_total ~ "ADI-R domain A",
        adi_r_comm_b_non_verbal_total ~ "ADI-R domain B",
        adi_r_rrb_c_total ~ "ADI-R domain C",
        vineland_ii_composite_standard_score ~ "VABS-II composite",
        ssc_diagnosis_nonverbal_iq ~ "Nonverbal IQ"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Demographic characteristics of study participants**") |>
    add_p() |>
    as_gt() |>
    gtsave(here("output","test_table.html"))

# Maternal WB5-HT by class membership

In [ ]:
df_core <- df_core |> mutate(class_3 = if_else(class == 3, 1, 0))

## Wilcoxon rank sum test

In [ ]:
library(exactRankTests)

In [ ]:
wilcox.exact(wb5ht_maternal ~ class_3, df_core)

## Harrell-Davis quantile estimator

In [ ]:
class_3_vs_non <- qcomhd(wb5ht_maternal ~ class_3, df_core)

In [ ]:
class_3_vs_non_table <- class_3_vs_non$partable |>
    select(-c(p.crit)) |>
    gt() |>
    tab_header(
        title = "Maternal WB5-HT by quantile for probands in class 3 compared to class 1 or 2") |>
    cols_label(
        q = "QU",
        n1 = md("*N* (Class 1 or 2)"),
        n2 = md("*N* (Class 3)"),
        est1 = "Median (Class 1 or 2)",
        est2 = "Median (Class 3)",
        "est1-est.2" = "Difference",
        ci.low = "CI Lower",
        ci.up = "CI Upper",
        p.value = md("*Adj. p*")) |>
    fmt_number(
        columns = 4:8,
        decimals = 2) |>
    fmt_number(
        columns = 9,
        decimals = 3) |>
    cols_align(
        align = "right",
        columns = everything()) |>
    tab_style(
        style = cell_text(weight = "bold"),
        locations = cells_body(
            rows = p.value < 0.05))

In [ ]:
class_3_vs_non_table |> gtsave("class_3_vs_non.html")